In [1]:
pip install mistralai

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/75.0 kB ? eta -:--:--
   --------------------- ------------------ 41.0/75.0 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 75.0/75.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-core 0.10.25.post3 requires openai>=1.1.0, but you have openai 0.27.9 which is incompatible.
llama-index-legacy 0.9.48 requires openai>=1.1.0, but you have openai 0.27.9 which is incompatible.
llama-index-llms-huggingface 0.1.4 requires huggingface-hub<0.21.0,>=0.20.3, but you have huggingface-hub 0.22.2 which is incompatible.


In [2]:
function_descriptions = [
    {
        "type": "function",
        "function":{
          "name": "move_group",
          "description": "Gives the transalational(metres) movements and angular(radians) rotations in x,y and z axis.",
          "parameters": {
              "type": "object",
              "properties": {
                  "x": {
                      "type": "object",
                      "properties": {
                          "value":{
                              "type": "integer",
                              "description": "Movement along x axis, the movement can be in +x axis(right) or in -x axis(left). "
                              },
                          "is_relative":{
                              "type": "boolean",
                              "description": "Specifies if the x-axis movement is relative"
                              }
                          }
                      },
                  "y": {
                      "type": "object",
                      "properties": {
                          "value":{
                              "type": "integer",
                              "description": "Movement along y axis, the movement can be in +y axis(forward) or in -y axis(backward)"
                              },
                          "is_relative":{
                              "type": "boolean",
                              "description": "Specifies if the y-axis movement is relative"
                              }
                          }
                      },
                  "z": {
                      "type": "object",
                      "properties": {
                          "value":{
                              "type": "integer",
                              "description": "Movement along z axis, the movement can be in +z axis(up) or in -z axis(down)"
                              },
                          "is_relative":{
                              "type": "boolean",
                              "description": "Specifies if the z-axis movement is relative"
                              }
                          }
                      },
                  "roll": {
                      "type": "object",
                      "properties": {
                          "value":{
                              "type": "integer",
                              "description": "Rotation along x-axis(longitudinal axis), the movement can be in +x axis(positive) or in -x axis(negative)"
                              },
                          "is_relative":{
                              "type": "boolean",
                              "description": "Specifies if the rotation in x-axis is relative"
                              }
                          }
                      },
                  "pitch": {
                      "type": "object",
                      "properties": {
                          "value":{
                              "type": "integer",
                              "description": "Rotation along y-axis(lateral axis), the movement can be in +y axis(positive/front) or in -y axis(negative/back)"
                              },
                          "is_relative":{
                              "type": "boolean",
                              "description": "Specifies if the rotation in y-axis is relative"
                              }
                          }
                      },
                  "yaw": {
                      "type": "object",
                      "properties": {
                          "value":{
                              "type": "integer",
                              "description": "Rotation along z-axis(vertical axis), the movement can be in clockwise(-ve) or anti-clockwise(+ve)"
                              },
                          "is_relative":{
                              "type": "boolean",
                              "description": "Specifies if the rotation in z-axis is relative"
                              }
                          }
                      },
                  }

              }
          }

    }
]

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import json
api_key=''

# model = "mistral-large-latest"
model = "open-mixtral-8x22b"

client = MistralClient(api_key=api_key)

def chat_mistral(prompt: str):
    call_id = 'yxBO5EvnL'
    exemplar_in = "move left"
    exemplar_out = {
        "x": {
            "value": -0.05,
            "is_relative": True
        }
    }

    function_info = function_descriptions[0]["function"]

    messages = [
        {'role': 'user', 'content': "move left"},
        {
            'role': 'assistant',
            'tool_calls': [{
                'id': call_id,
                'function': {
                    'name': function_info["name"],
                    'arguments': json.dumps(exemplar_out)
                },
                'type': 'function'
            }],
        },
        {
            "role": "tool",
            "tool_call_id": call_id,
            "name": function_info["name"],
            "content": json.dumps(exemplar_out),
        },
        {
            'role': 'assistant',  # Add an assistant message to handle output from tool before user continues
            'content': 'The move has been processed.'  # Placeholder text, adjust based on actual use case
        },
        {'role': 'user', 'content': prompt},
    ]

    chat_response = client.chat(
        model=model,
        messages=messages,
        tools=function_descriptions,
        tool_choice="auto"
    )

    return chat_response.choices[0].message

In [6]:
prompt = """move up"""
print(chat_mistral(prompt))

role='assistant' content='' name=None tool_calls=[ToolCall(id='rUYScyGC9', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": 0.05, "is_relative": true}}'))]


In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import json

api_key = ''
model = "open-mixtral-8x22b"
client = MistralClient(api_key=api_key)

def chat_mistral(prompt: str):
    # Define multiple exemplars
    exemplars = [
        ("move left", {"x": {"value": -0.05, "is_relative": True}}),
        ("move slightly up", {"z": {"value": 0.025, "is_relative": True}}),
        ("x=2m", {"x": {"value": 2, "is_relative": False}}),
        ("turn right by 45 degrees", {"yaw": {"value": 0.785398, "is_relative": True}}),
        ("tilt left", {"roll": {"value": -0.1745, "is_relative": True}}),
        ("tilt slightly back", {"pitch": {"value": -0.0872, "is_relative": True}}),
        ("turn left slightly and roll to the right", {"yaw": {"value": -0.0872, "is_relative": True}, "roll": {"value": 0.1745, "is_relative": True}}),
        ("move to x=2 and pitch up", {"x": {"value": 2, "is_relative": False}, "pitch": {"value": 0.1745, "is_relative": True}}),
        ("take 10m forward, x=10cm", {"y": {"value": 10, "is_relative": True}, "x": {"value": 0.1, "is_relative": False}}),
        ("take left and go slightly up", {"x": {"value": -0.05, "is_relative": True}, "z": {"value": 0.025, "is_relative": True}}),
        ("move 0.7m back-right with relative yaw angle of 36 degree", {"y": {"value": -0.7, "is_relative": True}, "x": {"value": 0.7, "is_relative": True},"yaw": {"value": 0.628, "is_relative": True}}),
        ("pitch up and roll =20 degrees", {"pitch": {"value": 0.1745, "is_relative": True}, "roll": {"value": 0.3490, "is_relative": False}}),
        ("turn slightly left and move up", {"pitch": {"value": -0.0872, "is_relative": True}, "z": {"value": 0.05, "is_relative": True}}),
        ("go back and turn right", {"y": {"value": -0.05, "is_relative": True}, "yaw": {"value": 0.0872, "is_relative": True}}),
    ]

    # Initialize messages list with exemplars
    messages = []
    for idx, (ex_in, ex_out) in enumerate(exemplars):
        #call_id = f'exemplar{idx}'
        call_id = f'exemplar1'
        function_info = function_descriptions[0]["function"]

        messages.extend([
            {'role': 'user', 'content': ex_in},
            {
                'role': 'assistant',
                'tool_calls': [{
                    'id': call_id,
                    'function': {
                        'name': function_info["name"],
                        'arguments': json.dumps(ex_out)
                    },
                    'type': 'function'
                }],
            },
            {
                "role": "tool",
                "tool_call_id": call_id,
                "name": function_info["name"],
                "content": json.dumps(ex_out),
            },
            {
                'role': 'assistant',
                'content': f'Response to {ex_in} processed.'
            }
        ])

    # Add the user's actual prompt at the end of the messages list
    messages.append({'role': 'user', 'content': prompt})
    #print(messages)

    # Make the chat call
    chat_response = client.chat(
        model=model,
        messages=messages,
        tools=function_descriptions,
        tool_choice="auto"
    )

    return chat_response.choices[0].message.tool_calls

In [16]:
prompt = """turn left slightly and roll to the right"""
print(chat_mistral(prompt))

[ToolCall(id='ystrDEjL9', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"yaw": {"value": -0.0872, "is_relative": true}, "roll": {"value": 0.1745, "is_relative": true}}'))]


In [54]:
prompt = """y=-2m""" #correct
print(chat_mistral(prompt))

[ToolCall(id='TR9OP4Myr', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": -2, "is_relative": false}}'))]


In [55]:
prompt = """y=2m, x=-2m""" #correct
print(chat_mistral(prompt))

[ToolCall(id='Db0q4bkRr', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 2, "is_relative": false}, "x": {"value": -2, "is_relative": false}}'))]


In [56]:
prompt = """move slightly up"""#correct
print(chat_mistral(prompt))

[ToolCall(id='S7bU6Lytg', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": 0.025, "is_relative": true}}'))]


In [57]:
prompt = """z=-12m"""#correct
print(chat_mistral(prompt))

[ToolCall(id='zQE6x08Ko', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": -12, "is_relative": false}}'))]


In [58]:
prompt = """Take the arm slightly left"""#correct
print(chat_mistral(prompt))

[ToolCall(id='K4eTe67m3', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": -0.025, "is_relative": true}}'))]


In [81]:
prompt = """tilt the manipulator to 30 degrees left"""#correct
print(chat_mistral(prompt))

[ToolCall(id='XJwBeJtT0', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"roll": {"value": -0.523599, "is_relative": true}}'))]


In [80]:
prompt = """Move the arm down from the current position"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='bzEb6Vz9s', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": -0.1, "is_relative": true}}'))]


In [61]:
prompt = """z=7m, x=5m""" #corret
print(chat_mistral(prompt))

[ToolCall(id='EGFVrZUKX', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": 5, "is_relative": false}, "z": {"value": 7, "is_relative": false}}'))]


In [62]:

prompt = """x=5m, y=10m"""#correct
print(chat_mistral(prompt))


[ToolCall(id='Vy5qbSDbP', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": 5, "is_relative": false}, "y": {"value": 10, "is_relative": false}}'))]


In [63]:
prompt = """go to x=0.5"""#correct
print(chat_mistral(prompt))

[ToolCall(id='wnuZFsDSD', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": 0.5, "is_relative": false}}'))]


In [79]:
prompt = """Move 10m forward""" #correct
print(chat_mistral(prompt))

[ToolCall(id='JVc3hHKjY', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 10, "is_relative": true}}'))]


In [65]:
prompt = """move up by 0.2m"""#correct
print(chat_mistral(prompt))

[ToolCall(id='xqaXhmJc1', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": 0.2, "is_relative": true}}'))]


In [78]:
prompt = """go forward slightly""" #incorrect
print(chat_mistral(prompt))

[ToolCall(id='Qg7bTmok0', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 0.05, "is_relative": true}}'))]


In [67]:
prompt = """go back by 0.2m""" #correct
print(chat_mistral(prompt))

[ToolCall(id='eHDN5SDd0', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": -0.2, "is_relative": true}}'))]


In [68]:
prompt = """go to z=-0.5""" #correct
print(chat_mistral(prompt))

[ToolCall(id='qkRMeqqPl', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": -0.5, "is_relative": false}}'))]


In [69]:
prompt = """go to z=-0.2 and x=0.1"""#correct
print(chat_mistral(prompt))

[ToolCall(id='xeOxDJV14', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": -0.2, "is_relative": false}, "x": {"value": 0.1, "is_relative": false}}'))]


In [70]:
prompt = """Move the arm 5m towards left"""#correct
print(chat_mistral(prompt))

[ToolCall(id='JY4xqbKmC', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": -5, "is_relative": true}}'))]


In [71]:
prompt = """take 0.1m left"""#correct
print(chat_mistral(prompt))

[ToolCall(id='lSxWMtkRA', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": -0.1, "is_relative": true}}'))]


In [77]:
prompt = """go front"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='3RrOBhniG', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 0.1, "is_relative": true}}'))]


In [76]:
prompt = """move slightly left"""#incorrct
print(chat_mistral(prompt))

[ToolCall(id='hFhL9hyRw', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": -0.05, "is_relative": true}}'))]


In [75]:
prompt = """move right by 0.1m"""#correct
print(chat_mistral(prompt))

[ToolCall(id='AMhiDwRBJ', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": 0.1, "is_relative": true}}'))]


In [83]:
prompt = """rotate to the left by 10 degrees"""#correct
print(chat_mistral(prompt))

[ToolCall(id='oT1IlnUch', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"yaw": {"value": -0.1745, "is_relative": true}}'))]


In [89]:
prompt = """yaw left slightly""" #correct
print(chat_mistral(prompt))

[ToolCall(id='olX6QmD5I', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"yaw": {"value": -0.0872, "is_relative": true}}'))]


In [90]:
prompt = """pitch upwards by 5 degrees"""#correct
print(chat_mistral(prompt))

[ToolCall(id='nQaZFpILk', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"pitch": {"value": 0.0872, "is_relative": true}}'))]


In [91]:
prompt = """pitch=40 degrees"""#correct
print(chat_mistral(prompt))

[ToolCall(id='BPWYD8eJ4', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"pitch": {"value": 0.698, "is_relative": false}}'))]


In [92]:
#####LINEAR LINEAR####

In [95]:
prompt = """move to y=0.2, x=0.1""" #corretc
print(chat_mistral(prompt))

[ToolCall(id='mqJSMqZro', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 0.2, "is_relative": false}, "x": {"value": 0.1, "is_relative": false}}'))]


In [99]:
prompt = """move 10m forward, take 20m left"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='HVwmAHUS5', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 10, "is_relative": false}, "x": {"value": -20, "is_relative": true}}'))]


In [102]:
prompt = """go straight for 0.3m and move right"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='IyYfuetJs', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 0.3, "is_relative": true}, "x": {"value": 0.1, "is_relative": true}}'))]


In [104]:
prompt = """come back and take a slight left"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='7wFD0lmp0', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"x": {"value": -1, "is_relative": true}, "yaw": {"value": -0.1745, "is_relative": true}}'))]


In [93]:
#####ANGULAR - ANGULAR####

In [106]:
prompt = """Tilt the manipulator arm forward by 30 degrees and turn right"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='K8RWcQMNL', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"pitch": {"value": 0.523599, "is_relative": true}}')), ToolCall(id='G0Qi9Px6q', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"yaw": {"value": -0.785398, "is_relative": true}}'))]


In [107]:
prompt = """Turn left slightly and roll to the right"""#correct
print(chat_mistral(prompt))

[ToolCall(id='9KJchkDrE', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"yaw": {"value": -0.0872, "is_relative": true}, "roll": {"value": 0.1745, "is_relative": true}}'))]


In [111]:
prompt = """roll slightly to the rigt and pitch downwards"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='vPYT4vXLA', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"roll": {"value": 0.0872, "is_relative": true}, "pitch": {"value": -0.0872, "is_relative": true}}'))]


In [110]:
prompt = """tilt right by 10 degrees and yaw left"""#correct
print(chat_mistral(prompt))

[ToolCall(id='8biRD1h3n', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"roll": {"value": 0.1745, "is_relative": true}, "yaw": {"value": -0.1745, "is_relative": true}}'))]


In [94]:
#####LINEAR - ANGULAR####

In [114]:
prompt = """Turn right and go to x=0.5"""#incorect
print(chat_mistral(prompt))

[ToolCall(id='JNA9WckJi', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"roll": {"value": 1.5707, "is_relative": true}, "x": {"value": 0.5, "is_relative": false}}'))]


In [115]:
prompt = """Move 0.1m front left with a relative yaw angle of 72 degrees """#correct
print(chat_mistral(prompt))

[ToolCall(id='WNfFQ6Us2', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": 0.1, "is_relative": true}, "x": {"value": -0.1, "is_relative": true}, "yaw": {"value": 1.2566, "is_relative": true}}'))]


In [118]:
prompt = """move 0.1m back-right with relative yaw angle of 72 degrees"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='dzIrkzUjs', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"y": {"value": -0.1, "is_relative": true}, "x": {"value": -0.07, "is_relative": true}, "yaw": {"value": 1.25664, "is_relative": true}}'))]


In [119]:
prompt = """move to z=2 and tilt left"""#correct
print(chat_mistral(prompt))

[ToolCall(id='yz18IYBGM', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": 2, "is_relative": false}, "roll": {"value": -0.1745, "is_relative": true}}'))]


In [122]:
prompt = """tilt slightly front and roll=10"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='Lh7g4ZmVf', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"pitch": {"value": -0.0872, "is_relative": true}, "roll": {"value": 0.1745, "is_relative": false}}'))]


In [125]:
prompt = """Yaw left and go up by 0.2m’"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='Ztjlpjy67', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"yaw": {"value": -0.349, "is_relative": true}, "z": {"value": 0.2, "is_relative": true}}'))]


In [129]:
prompt = """come down and yaw=10 degrees"""#incorrect
print(chat_mistral(prompt))

[ToolCall(id='lYSUZtTlD', type=<ToolType.function: 'function'>, function=FunctionCall(name='move_group', arguments='{"z": {"value": -0.2, "is_relative": true}, "yaw": {"value": 0.1745, "is_relative": true}}'))]
